In [6]:
import geopandas as gpd
import pandas as pd

from shapely.geometry import Point
from pathlib import Path
from typing import Dict

## Funções

In [ ]:
def extrair_lat_lon(caminho_csv: Path) -> Dict[str, float]:
    """
    Extrai Latitude e Longitude do arquivo csv.

    Parameters
    ----------
    caminho_csv : Path
        Caminho do arquivo csv.

    Returns
    -------
    dict[str, float]
        Dicionário contendo latitude e longitude.
    """
    try:
        with open(caminho_csv, encoding="latin1") as f:
            linhas = [next(f).strip() for _ in range(8)]

        metadados = {}

        for linha in linhas:
            if ";" in linha:
                chave, valor = linha.split(";", 1)
                metadados[chave.strip()] = valor.strip()

        latitude = float(metadados.get("LATITUDE:", "nan").replace(",", "."))
        longitude = float(metadados.get("LONGITUDE:", "nan").replace(",", "."))
        codigo = str(metadados.get("CODIGO (WMO):", "nan"))

        dict = {
            "latitude": latitude,
            "longitude": longitude,
            "codigo": codigo
            }

        return dict
    
    except Exception as erro:
        print(f"Não foi possível obter lat e lon de {caminho_csv}.")
        print(erro)


def filtrar_postos_da_bacia(
    df: pd.DataFrame, shp: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    """
    Filtra os postos do inventário a partir do contorno shapefile desejado.

    Parameters
    ----------
    df : pd.Dataframe
        Dataframe do inventário ANA, vinda da função obter_inventario().
    shp : gpd.GeoDataFrame
        Contorno desejado.

    Returns
    -------
    gpd.GeoDataFrame
        Postos que estão dentro do contorno.
    """
    df_inventario = df.copy()

    geometria = [
        Point(xy) for xy in zip(df_inventario["longitude"], df_inventario["latitude"])
    ]
    gdf_inventario = gpd.GeoDataFrame(
        df_inventario, geometry=geometria, crs="EPSG:4326"
    )

    if shp.crs != gdf_inventario.crs:
        shp = shp.set_crs(gdf_inventario.crs, allow_override=True)

    gdf_inventario = gdf_inventario[
        gdf_inventario.geometry.within(shp.geometry.union_all())
    ]

    return gdf_inventario

## Lógica Para organizar os dados em pastas de bacias

In [2]:
caminho_csvs = Path("/home/rodrigocaldas/dados_tcc/chuva_obs_inmet")
caminho_bacias = Path("/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/sub-bacias-isoladas")
dados_filtrados = Path("dados-filtrados")

caminhos_csvs = list(caminho_csvs.rglob("*.CSV"))
caminhos_bacias = list(caminho_bacias.rglob("*.shp"))

In [ ]:
for caminho_bacia in caminhos_bacias:
    for caminho_csv in caminhos_csvs:
        lat_lon_cod = extrair_lat_lon(caminho_csv=caminho_csv)

        gdf = gpd.read_file(caminho_bacia)
        df = pd.read_csv(caminho_csv, sep=';', skiprows=8, encoding='latin1')
        df = df.drop(df.columns[3::], axis=1)

        df["latitude"] = lat_lon_cod["latitude"]
        df["longitude"] = lat_lon_cod["longitude"]
        df["codigo"] = lat_lon_cod["codigo"]

        gdf_filtrado = filtrar_postos_da_bacia(df, gdf)

        if not gdf_filtrado.shape[0] > 0 or gdf_filtrado["Data"].isnull().all():
            pass

        else:
            caminho_dados_filtrados = Path(dados_filtrados, caminho_bacia.name)
            caminho_dados_filtrados.mkdir(parents=True, exist_ok=True)
            gdf_filtrado.to_csv(f"{caminho_dados_filtrados}/{caminho_csv.name}")
            #caminho_csv.unlink()

            print(f"Estação {caminho_csv.name} pertence a bacia {caminho_bacia.name}!")

## Organizar os dados por códigos nas pastas

In [7]:
caminho = Path("/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados")
pastas_bacias = caminho.rglob("*.CSV")

In [8]:
lista_codigo_inmet = []

for pasta in pastas_bacias:
    nome_csv = pasta.stem
    nome_csv_separado = nome_csv.split("_")
    codigo = nome_csv_separado[3]
    lista_codigo_inmet.append(codigo)

lista_sem_duplicatas = list(dict.fromkeys(lista_codigo_inmet))

In [21]:
pastas_bacias = caminho.rglob("*.CSV")
lista_gdf = []
for csv in pastas_bacias:
    nome_csv = csv.stem
    print(csv)

/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/INMET_CO_MS_A721_DOURADOS_01-01-2020_A_31-12-2020.CSV
/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/INMET_CO_MS_A731_MARACAJU_01-01-2021_A_31-12-2021.CSV
/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/INMET_CO_MS_A721_DOURADOS_01-01-2022_A_31-12-2022.CSV
/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/INMET_CO_MS_A709_IVINHEMA_01-01-2020_A_31-12-2020.CSV
/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/INMET_CO_MS_S712_NOVA ALVORADA DO SUL_01-01-2019_A_31-12-2019.CSV
/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/INMET_CO_MS_S711_LAGUNA CARAPA_01-01-2021_A_31-12-2021.CSV
/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/INMET_CO_MS_A754_SIDROLANDIA_01-01-2021_A_31-12-2021.CSV
/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados/ivinhema.shp/IN

In [30]:
for codigo in lista_sem_duplicatas:
    pastas_bacias = caminho.rglob("*.CSV")
    lista_gdf = []
    for csv in pastas_bacias:
        nome_csv = csv.stem
        nome_csv_separado = nome_csv.split("_")
        codigo_csv = nome_csv_separado[3]
        if codigo == codigo_csv:
            gdf = gpd.read_file(csv)
            gdf["Data"] = pd.to_datetime(gdf["Data"], format="%Y/%m/%d")
            gdf = gdf.drop(columns="field_1").set_index("Data")
            lista_gdf.append(gdf)
            csv.unlink()

    gdf_concatenados = pd.concat(lista_gdf, axis=0)
    gdf_concatenados = gdf_concatenados.sort_index()
    gdf_concatenados.to_csv(f"dados-organizados/{codigo}.csv")

print("Fim do serviço")

Fim do serviço


In [55]:
caminho_csvs = Path("/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-organizados")
caminho_bacias = Path("/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/sub-bacias-isoladas")
dados_filtrados = Path("dados-filtrados")

caminhos_csvs = list(caminho_csvs.rglob("*.csv"))
caminhos_bacias = list(caminho_bacias.rglob("*.shp"))

In [61]:
for caminho_bacia in caminhos_bacias:
    caminhos_csvs = list(caminho_csvs.rglob("*.csv"))
    for caminho_csv in caminhos_csvs:
        gdf = gpd.read_file(caminho_bacia)
        #df = gpd.read_csv(caminho_csv).drop(columns="geometry")
        df = pd.read_csv(caminho_csv, sep=',', encoding='latin1')
        #df = df.drop(df.columns[3::], axis=1)

        gdf_filtrado = filtrar_postos_da_bacia(df, gdf)

        if not gdf_filtrado.shape[0] > 0 or gdf_filtrado["Data"].isnull().all():
            pass

        else:
            caminho_dados_filtrados = Path(dados_filtrados, caminho_bacia.stem)
            caminho_dados_filtrados.mkdir(parents=True, exist_ok=True)
            gdf_filtrado.to_csv(f"{caminho_dados_filtrados}/{caminho_csv.name}")
            #caminho_csv.unlink()

            print(f"Estação {caminho_csv.name} pertence a bacia {caminho_bacia.name}!")

Estação A731.csv pertence a bacia ivinhema.shp!
Estação S708.csv pertence a bacia ivinhema.shp!
Estação A721.csv pertence a bacia ivinhema.shp!
Estação A709.csv pertence a bacia ivinhema.shp!
Estação S701.csv pertence a bacia ivinhema.shp!
Estação S711.csv pertence a bacia ivinhema.shp!
Estação S712.csv pertence a bacia ivinhema.shp!
Estação A743.csv pertence a bacia ivinhema.shp!
Estação A754.csv pertence a bacia ivinhema.shp!
Estação S710.csv pertence a bacia ivinhema.shp!
Estação S713.csv pertence a bacia ivinhema.shp!
Estação A703.csv pertence a bacia ivinhema.shp!
Estação B803.csv pertence a bacia balsa_santa_maria.shp!
Estação A825.csv pertence a bacia balsa_santa_maria.shp!
Estação S706.csv pertence a bacia florida_estrada.shp!
Estação S702.csv pertence a bacia florida_estrada.shp!
Estação S709.csv pertence a bacia florida_estrada.shp!
Estação A749.csv pertence a bacia florida_estrada.shp!
Estação A750.csv pertence a bacia florida_estrada.shp!
Estação A751.csv pertence a bacia f

## Quantos CSVS tem nas pastas?

In [6]:
from pathlib import Path
import pandas as pd

caminho_raiz = Path("/home/rodrigocaldas/dados_tcc/chuva_obs_inmet/dados-filtrados")  # Substitua pelo caminho correto

dados = []
for pasta in caminho_raiz.iterdir():
    if pasta.is_dir():
        numero_csvs = len(list(pasta.glob("*.csv")))
        dados.append({"pasta": pasta.name, "quantidade_csv": numero_csvs})

df = pd.DataFrame(dados)

df

,pasta,quantidade_csv
0,principe_da_beira,3
1,foz_do_rio_claro,3
2,pedra_do_cavalo,3
3,serra_da_mesa,2
4,foz_do_areia,2
...,...,...
81,irape,1
82,conceicao_do_araguaia,14
83,passagem,2
84,marimbondo,4
